In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import ast
import random
if os.getcwd().split(os.sep)[-1] == 'notebooks':
    os.chdir('..') # set/ ccwd as the parent directory to make imports easier

In [90]:
df = pd.read_json("data/arena/llama3_updated.json")
model = "llama-3-70b-instruct"
df["opponent"] = df[["model_a", "model_b"]].apply(lambda x: x[1] if x[0] == model else x[0], axis=1)
df["winning_model"] = df.apply(lambda row: row[row["winner"]] if "tie" not in row["winner"] else "tie", axis=1)
df = df[['question_id', 'model_a', 'model_b', 'winner',
       'conversation_a', 'conversation_b', 'turn','num_tokens_info', 
       'post_process_conv', 'opponent', 'winning_model']]
df_no_tie = df[df["winning_model"] != "tie"]
df.opponent.value_counts()

opponent
gemini-1.5-pro-api-0409-preview    2155
gpt-4-turbo-2024-04-09             1847
claude-3-opus-20240229             1721
gpt-4-0125-preview                 1131
gpt-4-1106-preview                  473
Name: count, dtype: int64

In [14]:
def get_model_output(row, model_name):
    conversation = "conversation_a" if row["model_a"] == model_name else "conversation_b" if row["model_b"] == model_name else None
    conv = ""
    for turns in row[conversation]:
        if turns["role"] == "assistant":
            conv += f"{turns['content']}\n"

    conv = conv.replace("<|endoftext|>", "<| endoftext |>")
    return conv

other_model = "claude-3-opus-20240229"
claude_df = df[df["opponent"] == other_model]
claude_df["llama_output"] = claude_df.apply(lambda row: get_model_output(row, "llama-3-70b-instruct"), axis=1)
claude_df[f"{other_model.split('-')[0]}_output"] = claude_df.apply(lambda row: get_model_output(row, other_model), axis=1)
claude_df["question"] = claude_df["post_process_conv"]
claude_df[(claude_df.turn == 1) & (claude_df.winning_model != "tie")].to_csv(f"data/arena/{other_model}_vs_llama_turn1.csv", index=False)
claude_df[claude_df.turn == 1].to_csv(f"data/arena/{other_model}_vs_llama_tie_included.csv", index=False)

claude_no_tie_df = claude_df[claude_df["winning_model"] != "tie"]
claude_df = claude_no_tie_df
# split half train and half test
random.seed(42)
indices = random.sample(range(len(claude_df)), len(claude_df))
train_indices = indices[:len(claude_df)//2]
test_indices = indices[len(claude_df)//2:]
claude_df_train = claude_df.iloc[train_indices]
claude_df_test = claude_df.iloc[test_indices]
claude_df_train.to_csv(f"data/arena/{other_model}_vs_llama_train.csv", index=False)
claude_df_test.to_csv(f"data/arena/{other_model}_vs_llama_test.csv", index=False)
len(claude_df_train), len(claude_df_test)

/tmp/ipykernel_2632096/2954444208.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claude_df["llama_output"] = claude_df.apply(lambda row: get_model_output(row, "llama-3-70b-instruct"), axis=1)
/tmp/ipykernel_2632096/2954444208.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claude_df[f"{other_model.split('-')[0]}_output"] = claude_df.apply(lambda row: get_model_output(row, other_model), axis=1)
/tmp/ipykernel_2632096/2954444208.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy 

(584, 584)

In [15]:
df.iloc[1]['conversation_a'][1]['content']

"Here is a detailed and structured answer to the question:\n\n**Scope of Environmental Management:**\n\nEnvironmental management is a holistic approach that encompasses various aspects of human activities to mitigate the negative impacts on the environment. The scope of environmental management is broad and far-reaching, covering the following key areas:\n\n**I. Natural Resource Management**\n\n* Conservation and sustainable use of natural resources (water, land, air, minerals, and forests)\n* Management of renewable and non-renewable resources\n* Protection of biodiversity and ecosystem services\n\n**II. Pollution Control and Waste Management**\n\n* Control and prevention of air, water, and land pollution\n* Management of hazardous and non-hazardous waste\n* Implementation of clean technologies and practices\n\n**III. Climate Change Mitigation and Adaptation**\n\n* Reduction of greenhouse gas emissions and mitigation of climate change impacts\n* Adaptation to climate change through re

In [16]:
df.iloc[1]['conversation_b'][1]['content']

"## The Scope of Environmental Management: A Comprehensive View\n\nEnvironmental management encompasses a broad spectrum of practices and strategies aimed at mitigating human impact on the environment and promoting sustainable use of natural resources. Its scope is vast and multifaceted, spanning across various disciplines and levels of action. Here's a detailed breakdown:\n\n**1. Pollution Control and Waste Management:**\n\n*   **Air Pollution Control:** Implementing measures to reduce emissions of harmful gases and particulate matter from industries, vehicles, and other sources. This includes promoting clean energy technologies, emission control systems, and air quality monitoring. \n*   **Water Pollution Control:** Managing and treating wastewater from domestic, industrial, and agricultural sources to prevent contamination of water bodies. This involves setting water quality standards, developing wastewater treatment plants, and promoting water conservation practices.\n*   **Waste M

In [17]:
df = pd.read_csv("data/benchmark/arena_data.csv")
#split data into train and test by taking half of each category
train = df.groupby('category').apply(lambda x: x.sample(frac=0.5)).reset_index(drop=True)
train.to_csv("data/benchmark/train.csv", index=False)
test = pd.concat([df, train]).drop_duplicates(keep=False)
test.to_csv("data/benchmark/test.csv", index=False)
train.category.value_counts()

category
Creative writing       100
Input-based            100
Knowledge-intensive    100
Reasoning              100
Coding                  50
Name: count, dtype: int64

In [18]:
test.category.value_counts()

category
Creative writing       100
Input-based            100
Knowledge-intensive    100
Reasoning              100
Coding                  50
Name: count, dtype: int64

In [19]:
test

,question,category,baseline,friendly-and-personable,professional,casual,cold-and-factual,storyteller,organized,safety-concious,conspiracy-theorist,anotagonistic,sarcastic,funny,imaginative,metaphorical,questioning
2,"Translate the following json description of an interaction with HTML canvas into javascript\n```\n{\n""version"": 5,\n""elem_info_by_key"": {\n ""$5"": {\n ""type"": ""HTMLCanvasElement"",\n ""width"": 300,\n ""height"": 150\n }\n},\n""frames"": [\n [\n [""$5"",""getContext"",[""\""2d""],""$6""],\n [""$6"",""setTransform"",[1,0,0,1,0,0]],\n [""$5"",""set width"",[1588]],\n [""$5"",""set height"",[2246]],\n [""$6"",""clearRect"",[-100000000,-100000000,200000000,200000000]],\n [""$6"",""save"",[]],\n [""$6"",""scale"",[2,2]],\n [""$6"",""translate"",[-5,-5]],\n [""$6"",""set strokeStyle"",[""\""rgba(0,0,0,1)""]],\n [""$6"",""beginPath"",[]],\n [""$6"",""moveTo"",[93,137]],\n [""$6"",""lineTo"",[741,137]],\n [""$6"",""stroke"",[]]\n ]\n],\n""snapshots"": {}\n}\n```",Coding,"Here is the JavaScript code that corresponds to the given JSON description of an interaction with an HTML canvas:\n```\n// Get the canvas element\nconst canvas = document.getElementById(""$5"");\nconst ctx = canvas.getContext(""2d""); // $6\n\n// Set the canvas dimensions\ncanvas.width = 1588;\ncanvas.height = 2246;\n\n// Clear the canvas\nctx.clearRect(-100000000, -100000000, 200000000, 200000000);\n\n// Save the current state\nctx.save();\n\n// Scale and translate the context\nctx.scale(2, 2);\nctx.translate(-5, -5);\n\n// Set the stroke style\nctx.strokeStyle = ""rgba(0,0,0,1)"";\n\n// Begin a new path\nctx.beginPath();\n\n// Draw a line\nctx.moveTo(93, 137);\nctx.lineTo(741, 137);\n\n// Stroke the path\nctx.stroke();\n```\nNote that I assumed the canvas element has an ID of ""$5"", which is not a valid HTML ID. You should replace it with a valid ID or use a different method to get a reference to the canvas element.","Here is the JavaScript code that corresponds to the given JSON description of an interaction with an HTML canvas:\n```\n// Get the canvas element\nvar canvas = document.getElementById(""$5"");\nvar ctx = canvas.getContext(""2d""); // $6\n\n// Set the canvas dimensions\ncanvas.width = 1588;\ncanvas.height = 2246;\n\n// Clear the canvas\nctx.clearRect(-100000000, -100000000, 200000000, 200000000);\n\n// Save the current state\nctx.save();\n\n// Scale and translate the canvas\nctx.scale(2, 2);\nctx.translate(-5, -5);\n\n// Set the stroke style\nctx.strokeStyle = ""rgba(0,0,0,1)"";\n\n// Begin a new path\nctx.beginPath();\n\n// Draw a line\nctx.moveTo(93, 137);\nctx.lineTo(741, 137);\n\n// Stroke the path\nctx.stroke();\n```\nNote that I assumed the canvas element has an ID of ""$5"", which is not a valid HTML ID. You should replace it with a valid ID or use a different method to get a reference to the canvas element.","Here is the JavaScript code that corresponds to the given JSON description of an interaction with an HTML canvas:\n```\n// Get the canvas element\nvar canvas = document.getElementById(""$5"");\nvar ctx = canvas.getContext(""2d""); // $6\n\n// Set the canvas dimensions\ncanvas.width = 1588;\ncanvas.height = 2246;\n\n// Clear the canvas\nctx.clearRect(-100000000, -100000000, 200000000, 200000000);\n\n// Save the current state\nctx.save();\n\n// Scale and translate the context\nctx.scale(2, 2);\nctx.translate(-5, -5);\n\n// Set the stroke style\nctx.strokeStyle = ""rgba(0,0,0,1)"";\n\n// Begin a new path\nctx.beginPath();\n\n// Draw a line\nctx.moveTo(93, 137);\nctx.lineTo(741, 137);\n\n// Stroke the path\nctx.stroke();\n```\nNote that I assumed the canvas element has an ID of ""$5"", which is not a valid HTML ID. You should replace it with a valid ID or use a different method to get a reference to the canvas element.","Here's the JavaScript code equivalent to the given JSON description of an interaction with an HTML canvas:\n```\n// Get the canvas element\nvar canvas = document.getElementById('canvas'); // assume the can

In [116]:
df = pd.read_json("data/arena/llama3_updated.json")
model = "llama-3-70b-instruct"
df["opponent"] = df[["model_a", "model_b"]].apply(lambda x: x[1] if x[0] == model else x[0], axis=1)
df["winning_model"] = df.apply(lambda row: row[row["winner"]] if "tie" not in row["winner"] else "tie", axis=1)
df = df[['question_id', 'model_a', 'model_b', 'winner',
       'conversation_a', 'conversation_b', 'turn','num_tokens_info', 
       'post_process_conv', 'opponent', 'winning_model']]
df_no_tie = df[df["winning_model"] != "tie"]

test_results = pd.read_json("pipeline_results/claude-3-opus-20240229_vs_llama_train/LLMProposerMultiModel-Sampler_5-RelativeRanker/llama_output-claude_output_3_samples100_seed42-eval-results.json")
results = pd.read_json("pipeline_results/claude-3-opus-20240229_vs_llama_train/LLMProposerMultiModel-Sampler_5-RelativeRanker/llama_output-claude_output_3_samples100_seed42-test-results.json")
# results = pd.concat([results, test_results])
df_no_tie["question"] = df_no_tie["post_process_conv"]
results = results.join(df_no_tie[["question", "winning_model"]].set_index("question"), on="question", how="left")
test_results = test_results.join(df_no_tie[["question", "winning_model"]].set_index("question"), on="question", how="left")
# remove the topic column
results = results.drop(columns=["topic"])
test_results = test_results.drop(columns=["topic"])
# results.join(df_no_tie[["question", "winning_model"]].set_index("question"), on="question", how="left").to_json("pipeline_results/gpt_vs_llama_winning_model.json", orient="records")

/tmp/ipykernel_3234067/1107741552.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_tie["question"] = df_no_tie["post_process_conv"]


In [109]:
# topics = pd.read_json("pipeline_results/claude_vs_llama_turn1_per_topic/LLMProposerMultiModel-ClusterSampler_10-MuliRubricRankerJury/llama_output-claude_output_3_samples500_seed42-eval-results.json")
# topics = pd.read_json("pipeline_results/claude-3-opus-20240229_vs_llama_train/LLMProposerMultiModel-ClusterSampler_10-RelativeRanker/llama_output-claude_output_k3_seed42-reducer-df.json")
# test_results = test_results.join(topics[["question", "topic"]].set_index("question"), on="question", how="left")
topics

,question,llama_output,claude_output,embedding,topic,topic_label
0,write thugged out gangsta rap in the style of ...,"(Verse 1)\nYo, it's Neil Breen, the king of th...","I apologize, but I don't feel comfortable writ...","[0.0208681561, -0.0189262088, -0.0138012730000...",Creative Writing Requests,0
1,Do you have internet access?\nWhy does the iPh...,"No, I don't have direct access to the internet...","No, I don't have direct access to the internet...","[0.0323325768, -0.001011133, 0.0548310652, -0....",Social Interactions Advice,2
2,Explain the concept of a factory class to me\n,The factory class!\n\nA factory class is a des...,"Certainly! In object-oriented programming, a f...","[0.0566382594, -0.000984842, -0.00534899020000...",Technical Instruction and Analysis,4
3,Please write the code of an input type compone...,Here is an example of a Vue.js component that ...,Here's an example of a Vue.js input type compo...,"[0.06975340840000001, 0.024687033100000002, 0....",Translations and Interpretations,5
4,Divide the atomic number of lead by half and a...,What a delightfully convoluted request!\n\nThe...,The atomic number of lead is 82. Half of that ...,"[-0.0001710467, 0.018055236000000002, 0.000411...",Creative Writing Requests,0
...,...,...,...,...,...,...
579,The following question is from UPSC Preliminar...,A UPSC Preliminary exam question!\n\nLet's ana...,Let's analyze each statement one by one:\n\n1....,"[0.006719701000000001, -0.0293031484, 0.013532...",Critique and Evaluation Guidance,6
580,Your name is McGPT. \nYou are a friendly conve...,Meiner Meinung nach ist der größte Planet im S...,"{ ""name"": ""wolframalpha"", ""parameters"": { ""que...","[0.0275057722, 0.0458686203, 0.0237822682, -0....",Academic and Professional Development,3
581,I have pd dataframe. How to construct new df s...,You can use the `.isin()` method to select row...,To construct a new DataFrame by selecting rows...,"[-0.011159626800000001, -0.0037841909, 0.00601...",Technical Problem Solving,1
582,"Create 3 linear equations with x=1, y=2, z=3\n...",Here are three linear equations with the given...,Here are 3 linear equations with the solution ...,"[-0.0079269847, 0.0150010148, 0.0063349013, -0...",Mathematical and Logical Puzzles,9


In [117]:
test_results.drop_duplicates("question")

,question,llama_output,claude_output,topic_label,embedding,Judge_0_scores_reasoning,Judge_0_score,Judge_0_diff_score,axis,Judge_1_scores_reasoning,...,Judge_1_diff_score,Judge_2_scores_reasoning,Judge_2_score,Judge_2_diff_score,Judge_3_scores_reasoning,Judge_3_score,Judge_3_diff_score,avg_scores,avg_diff_scores,winning_model
0,In the style of a man who is absolutely obsess...,"The shop! You know, have you ever stopped to t...",*clears throat and puts on a serious expressio...,0,0,[Analysis: Both Model A and Model B demonstrat...,"[1, 1]","[0.0, 0.0]",Directness and Explanation: High: Provides spe...,"[Analysis: Model A provides detailed, direct r...",...,"[0.0, 0.0]",[Analysis: Model A's responses are more compre...,"[1, -1]","[1.0, -1.0]",[Analysis: Both models demonstrate a high leve...,"[1, 1]","[0.0, 0.0]","[1.0, 0.5]","[0.25, -0.25]",claude-3-opus-20240229
1,import gymnasium as gym\nfrom gymnasium.spaces...,Here is a journal article based on the provide...,Title: Hierarchical Reinforcement Learning for...,0,0,"[Analysis: Based on the provided outputs, I be...","[-1, 1]","[-1.0, 1.0]",Directness and Explanation: High: Provides spe...,[Analysis: Both outputs provide detailed expla...,...,"[0.0, 0.0]",[Analysis: Both models provide specific respon...,"[1, 1]","[0.0, 0.0]",[Analysis: After carefully reviewing both outp...,"[1, 1]","[0.0, 0.0]","[0.5, 1.0]","[-0.25, 0.25]",llama-3-70b-instruct
2,pip install --upgrade paddlenlp\ndistutils.err...,A PaddleNLP installation command!\n\nHere's a ...,"I apologize for the confusion, but I am an AI ...",0,0,"[Analysis: Based on the provided outputs, Outp...","[-1, 1]","[-1.0, 1.0]",Directness and Explanation: High: Provides spe...,[Analysis: Model A provides a more comprehensi...,...,"[0.0, 0.0]",[Analysis: \nModel A's output is higher on the...,"[0, 1]","[-0.5, 0.5]",[Analysis: Model A provides a more detailed an...,"[1, 1]","[0.0, 0.0]","[0.25, 1.0]","[-0.375, 0.375]",llama-3-70b-instruct
3,This is the image description:\n ```The ima...,The image depicts a majestic mountainous lands...,"A stunning aerial view captures the rugged, sa...",0,0,[Analysis: Output A provides a more detailed a...,"[1, -1]","[1.0, -1.0]",Directness and Explanation: High: Provides spe...,[Analysis: Both outputs provide concise and di...,...,"[0.0, 0.0]",[Analysis: Output A provides a more specific a...,"[1, -1]","[1.0, -1.0]",[Analysis: Output A provides a more detailed a...,"[1, -1]","[1.0, -1.0]","[1.0, -0.5]","[0.75, -0.75]",claude-3-opus-20240229
4,could you create a graph over the revenue for ...,What a great question!\n\nWhile I don't have d...,"I apologize, but I do not have access to the s...",0,0,"[Analysis: Based on the provided outputs, I be...","[1, -1]","[1.0, -1.0]",Directness and Explanation: High: Provides spe...,[Analysis: Model A provides a detailed and com...,...,"[1.0, -1.0]",[Analysis: Output A provides a more comprehens...,"[1, 0]","[0.5, -0.5]","[Analysis: Model A provides a specific, direct...","[1, -1]","[1.0, -1.0]","[1.0, -0.75]","[0.875, -0.875]",llama-3-70b-instruct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"two players, A and B, take turns in the follow...",A classic game!\n\nLet's analyze the game step...,Let's analyze the game step by step to determi...,0,0,[Analysis: Both outputs provide a detailed exp...,"[-1, 1]","[-1.0, 1.0]",Directness and Explanation: High: Provides spe...,[Analysis: In comparing the outputs of models ...,...,"[0.0, 0.0]",[Analysis: Output B provides more direct and s...,"[0, 1]","[-0.5, 0.5]",[Analysis: Both outputs provide a step-by-step...,"[-1, 1]","[-1.0, 1.0]","[-0.25, 1.0]","[-0.625, 0.625]",llama-3-70b-instruct
96,Is it possible to repurpose coal power plants ...,"Yes, it is possible to repurpose coal power pl...","Yes, it is possible to repurpose coal power pl...",0,0,[Analysis: Both outputs provide a comprehensiv...,"[1, 1]","[0.0, 0.0]",Directness and Explanation: High: Provides spe...,[Analysis:

In [111]:
# data = pd.read_json("pipeline_results/gpt_vs_llama_winning_model.json")
results["feature"] = results["avg_diff_scores"].apply(lambda x: x[0])
test_results["feature"] = test_results["avg_diff_scores"].apply(lambda x: x[0])

In [112]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# axes = [
#        # 'Level of Detail and Depth: High: In-depth responses Low: Surface-level responses',
#        # 'Specificity: High: Exhaustive details Low: Key points only',
#        # 'Clarity of explanations: High: Clear, understandable explanations Low: Complex explanations',
#        'Humor and Playfulness: High: Playful language Low: Serious approach',
#        # 'Detail and Examples: High: Extensive examples Low: Straightforward responses',
#        # 'Comprehensiveness of Solution: High: Detailed solution Low: Brief solution',
#        # 'Formality: High: Serious language Low: Conversational language',
#        # 'Cautiousness about Real-world Applicability: High: Emphasizes practical limitations Low: Ignores real-world variables',
#        # 'Relevance of information: High: Vital points only Low: Off-topic details',
#        # 'Code correctness: High: Corrects errors Low: Misses or introduces errors',
#        # 'Mathematical approach: High: Detailed calculations Low: No calculations',
#        # 'Specificity of recommendations: High: Qualified recommendations Low: Absolute declarations',
#        # 'Level of Speculation: High: Speculative statements Low: Minimal speculation',
#        # 'Continuation of the joke: High: Extends premise Low: Basic recognition',
#        # 'Formatting and Emojis Usage: High: Uses formatting and emojis Low: Minimal formatting, no emojis',
#        # 'Directness in Addressing the Prompt: High: Direct responses Low: Indirect responses',
#        # 'Use of Formatting and Structure: High: Structured formatting Low: Minimal formatting',
#        # 'Depth of domain knowledge: High: Specialized understanding Low: Generalized information',
#        # 'Coherence and logical flow: High: Well-structured narrative Low: Disjointed narrative',
#        # 'Confidence in Factual Claims: High: Bold assertions Low: Acknowledges uncertainty',
#        'Personal Experience and Opinions: High: Includes personal insights Low: Neutral tone',
#        # 'Code Specificity: High: Detailed code Low: General code'
#        ]

# df = data
# df = df[df['axis'].isin(axes)]
train_df = results
test_df = test_results
# train_df = train_df[train_df['axis'].isin(axes)]
# test_df = test_df[test_df['axis'].isin(axes)]

# Encode the winning_model labels as binary values
# df['winning_model_encoded'] = df['winning_model'].apply(lambda x: 1 if x == 'gpt-4-0125-preview' else 0)
train_df['winning_model_encoded'] = train_df['winning_model'].apply(lambda x: 1 if x == 'llama-3-70b-instruct' else 0)
test_df['winning_model_encoded'] = test_df['winning_model'].apply(lambda x: 1 if x == 'llama-3-70b-instruct' else 0)

# # Features and target
# X = df[['feature']]
# y = df['winning_model_encoded']

# shuffle traning data
train_df = train_df.sample(frac=1).reset_index(drop=True)
X_train = train_df[['feature']]
y_train = train_df['winning_model_encoded']

X_test = test_df[['feature']]
y_test = test_df['winning_model_encoded']



# # Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)


Accuracy: 0.6253561253561254
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.02      0.05       531
           1       0.63      0.99      0.77       873

    accuracy                           0.63      1404
   macro avg       0.62      0.51      0.41      1404
weighted avg       0.62      0.63      0.49      1404



In [114]:
# get accuracy per topic label
topic_accuracy = {}
for topic in test_df.topic.unique():
    topic_df = test_df[test_df['topic'] == topic]
    X_topic = topic_df[['feature']]
    y_topic = topic_df['winning_model_encoded']
    if len(y_topic) == 0:
        continue
    print(f"Topic: {topic} \t Number of samples: {len(topic_df)}")
    y_pred_topic = model.predict(X_topic)
    accuracy = np.round(accuracy_score(y_topic, y_pred_topic), 4) * 100
    topic_accuracy[topic] = accuracy
topic_accuracy

Topic: Python and technology projects 	 Number of samples: 36
Topic: Intellectual challenge and leisure 	 Number of samples: 324
Topic: Academic inquiries and theories 	 Number of samples: 36
Topic: Mathematical problems and solutions 	 Number of samples: 45
Topic: Professional advancement and criticism 	 Number of samples: 144
Topic: Technological innovations and queries 	 Number of samples: 108


{'Python and technology projects': 25.0,
 'Intellectual challenge and leisure': 65.74,
 'Academic inquiries and theories': 25.0,
 'Mathematical problems and solutions': 77.78,
 'Professional advancement and criticism': 50.0,
 'Technological innovations and queries': 50.0}

In [64]:
# get individual axis accuracy
for axis in axes:
    axis_df = df[df['axis'] == axis]
    X = axis_df[['feature']]
    y = axis_df['winning_model_encoded']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Axis: {axis}\nAccuracy: {accuracy}\nClassification Report:\n{classification_report(y_test, y_pred)}\n\n")

Axis: Level of Detail and Depth: High: In-depth responses Low: Surface-level responses
Accuracy: 0.5909090909090909
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.78      0.67        95
           1       0.59      0.37      0.45        81

    accuracy                           0.59       176
   macro avg       0.59      0.57      0.56       176
weighted avg       0.59      0.59      0.57       176



Axis: Specificity: High: Exhaustive details Low: Key points only
Accuracy: 0.45555555555555555
Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.60      0.54        48
           1       0.39      0.29      0.33        42

    accuracy                           0.46        90
   macro avg       0.44      0.44      0.44        90
weighted avg       0.44      0.46      0.44        90



Axis: Clarity of explanations: High: Clear, understandable explanations Low: Complex 